# Scale your Machine Learning Models for Faster Training with Sklearn Joblib

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
X, y = make_classification(n_samples=10_000_000, n_features=4,
                          n_informative=2, n_redundant=0,
                          random_state=0, shuffle=False)

In [3]:
X.size

40000000

In [16]:
import pandas as pd
dfX = pd.DataFrame(data=X)

In [18]:
dfX.to_csv("X_test.csv")

This makes `X` about XXMB.

In [4]:
clf = RandomForestClassifier(
    max_depth=2, 
    random_state=0, 
    n_jobs=-1
)

In [10]:
%%time
clf.fit(X,y)

CPU times: user 13min 21s, sys: 17.8 s, total: 13min 38s
Wall time: 2min 6s


RandomForestClassifier(max_depth=2, n_jobs=-1, random_state=0)

### Spin up Coiled Cluster

In [5]:
import coiled

In [6]:
cluster = coiled.Cluster(
    name="sklearn",
    software="dask-nlp",
    n_workers=20,
    scheduler_options={'idle_timeout': '2 hours'},
    backend_options={'spot':'True'}
)

Output()

Using existing cluster: 'sklearn'

In [7]:
from distributed import Client
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/dask-nlp/lib/python3.9/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+---------+----------------+---------------+---------------+
| Package | client         | scheduler     | workers       |
+---------+----------------+---------------+---------------+
| msgpack | 1.0.3          | 1.0.2         | 1.0.2         |
| python  | 3.9.10.final.0 | 3.9.7.final.0 | 3.9.7.final.0 |
+---------+----------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.4.10.86:8786' processes=20 threads=40, memory=153.34 GiB>

In [21]:
client.restart()

<Client: 'tls://10.4.10.86:8786' processes=18 threads=36, memory=138.01 GiB>

### Fit Model with Dask

In [22]:
import joblib

In [23]:
%%time
with joblib.parallel_backend("dask"):
    clf.fit(X, y)

CPU times: user 1.93 s, sys: 601 ms, total: 2.53 s
Wall time: 1min 1s


## GridSearch CV

In [24]:
from sklearn.model_selection import GridSearchCV
import numpy as np

In [25]:
# Create a parameter grid
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

In [26]:
# Instantiate the grid search model
grid_search = GridSearchCV(
    estimator=clf, 
    param_grid=param_grid, 
    cv=5, 
    n_jobs=-1
)

In [ ]:
%%time
with joblib.parallel_backend("dask"):
    grid_search.fit(X, y)